In [3]:
import tensorflow as tf

a = tf.placeholder(tf.float32, name='a')
b = tf.placeholder(tf.float32, name='b')
adder = tf.add(a, b)
add_and_triple = adder * 3
sess = tf.Session()
print(sess.run(add_and_triple, {a: 0.3, b: 0.7}))

3.0


In [4]:
w1 = tf.Variable([-0.3], tf.float32)
w2 = tf.Variable([0.3], tf.float32)

init = tf.global_variables_initializer()
sess.run(init)
print(sess.run((w1, w2)))

(array([-0.30000001], dtype=float32), array([ 0.30000001], dtype=float32))


# 示例：线性回归

y = Wx + b

训练样本：

[1, 0]
[2, -1]
[3, -2]
[4, -3]

W、b 如何找出来？

In [5]:
import tensorflow as tf

W = tf.Variable([.3], tf.float32, name='W')
b = tf.Variable([-.3], tf.float32, name='b')

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

x = tf.placeholder(tf.float32, name='x')
y = tf.placeholder(tf.float32, name='y')
linear_module = W * x + b

square_delta = tf.square(linear_module - y)
loss = tf.reduce_sum(square_delta)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run((W, b)))
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))
print(sess.run(linear_module, {x: [3]}))

(array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32))
5.69997e-11
[-1.99999988]


In [32]:
import numpy as np
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def


def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="http://7xj89i.com1.z0.glb.clouddn.com/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
        """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

graph = tf.Graph()
with graph.as_default():
    sess = tf.InteractiveSession(graph=graph)
    W = tf.Variable([.3], tf.float32, name='W')
    b = tf.Variable([-.3], tf.float32, name='b')

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    x = tf.placeholder(tf.float32, name='x')
    y = tf.placeholder(tf.float32, name='y')
    linear_module = W * x + b

    square_delta = tf.square(linear_module - y)
    loss = tf.reduce_sum(square_delta)

    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)

    for i in range(1000):
        sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}) 
    
    show_graph(graph)

In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})